In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"mostafaessaa","key":"216f07e46f5cb98940523b5edc0b0ec2"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

!kaggle datasets download -d snehaanbhawal/resume-dataset

!unzip /content/resume-dataset.zip


In [ ]:
!pip install skillNer

!python -m spacy download en_core_web_lg

!pip install PyPDF2

!pip install tqdm

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor
import pandas as pd 
from PyPDF2 import PdfReader
import re
from tqdm import tqdm
import os 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

nlp = spacy.load("en_core_web_lg")
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

In [ ]:
#function to prase text
def prase(text):
  annotations = skill_extractor.annotate(text)
  skills = []
  for i in range(len(annotations['results']['ngram_scored'])):
    skills.append(annotations['results']['ngram_scored'][i]['doc_node_value'])
  for i in range(len(annotations['results']['full_matches'])):
    skills.append(annotations['results']['full_matches'][i]['doc_node_value'])
    return list(set(skills))

In [ ]:
#function to convert pdf-file to text
def p2t(x):
  parts=[]
  text=''
  reader = PdfReader(x)
  for i in range(len(reader.pages)):
    page = reader.pages[i]
    parts.append(page.extract_text())
  text = ' '.join(parts)
  return text

In [ ]:
#function to clean text
def cleanText(Text):
    Text = re.sub('IDs','0',Text)
    Text = re.sub(r'[0-9]+', '', Text)
    Text = Text.split()
    rx = re.compile(r'\D*\d')
    Text = [x for x in Text if not rx.match(x)]
    Text = [x.lstrip() for x in Text]
    Text = ' '.join(Text)
    Text = re.sub(r'[^\x00-\x7F]+','0',Text)
    Text = re.sub(r'[\W_]', ' ', Text)
    Text = re.sub(r'(?<=\d) (?=\d)', '', Text)
    Text = re.sub('http\S+\s*', ' ', Text)  # remove URLs
    Text = re.sub('RT|cc', ' ', Text)  # remove RT and cc
    Text = re.sub('#\S+', '', Text)  # remove hashtags
    Text = re.sub('@\S+', '  ', Text)  # remove mentions
    Text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', Text)  # remove punctuations
    Text = re.sub('\s+', ' ', Text)  # remove extra whitespace
    return Text

In [ ]:
# funtion to turn pdf files to csv which will have column Skills
def pdf2csv(path):
  ser = []
  id = []
  x = os.listdir(path)
  for filen in tqdm(x):
    if filen.split('.')[1] == 'pdf':
      print(filen)
      m = os.path.join(path,filen)
      data = prase(cleanText(p2t(m)))
      ser.append(data)
      id.append(filen)
    else:
      pass
  for i in range(len(ser)):
    ser[i] = ' '.join(ser[i])
  data = pd.DataFrame(ser)
  data['Csv_id'] = id
  return data

In [ ]:
#still under construction
def fast_pdf_depend(filen):
    if filen.split('.')[1] == 'pdf':
      print(filen)
      m = os.path.join(path,filen)
      data = prase(cleanText(p2t(m)))
    else:
      pass
    return data,filen

In [ ]:
#still under construction
def Fast_pdf2csv(path,o=10):
  ser = []
  id = []
  pool = Pool(processes=o)
  g = os.listdir(path)
  k=pool.map_async(fast_pdf_depend,g)
  ser.append(k)
  if k.ready():
    if k.succesful():
      print('succesful')
    else:
     print('unsuccesful') 
  return ser

In [ ]:
#function to match text with data takes data['column'] as input
def dmatch(text,data):
  input = cleanText(text)
  input_skills = prase(text)
  input_skills = ' '.join(input_skills)
  vectorizer = TfidfVectorizer()
  vec1 = vectorizer.fit_transform(data)
  vec2 = vectorizer.transform([input_skills])
  sim_matrix1=[]
  for i in range(vec1.shape[0]):
    similarity = cosine_similarity(vec2.toarray(),vec1[i].toarray())
    sim_matrix1.append(similarity)
  data['match'] = np.round(np.ravel(sim_matrix1)*100)
  data.sort_values('match',ascending=False,inplace=True)
  return data

In [ ]:
#function to prase csv file which has resume as str takes path of .csv file and column of data which has str data
def csvprase(path,column):
  data = pd.read_csv(path)
  data['index1'] = data.index
  data['skills'] = data.column.apply(cleanText)
  data['skills'] = data['skills'].apply(prase)
  return data

In [ ]:
from multiprocessing import Pool

In [ ]:

path = '/content/cvs'
data = Fast_pdf2csv(path)

In [ ]:
data.get()

In [ ]:
text = '''Job Details
Experience Needed:
0 To 2 Years
Career Level:
Entry Level (Junior Level / Fresh Grad)
Education Level:
Bachelor's Degree
Salary:
Confidential
Job Categories:
Engineering - Mechanical/Electrical
Skills And Tools:
Electrical Engineer
Electrical Engineering
Job Description
Providing assistance to ensure all electrical and instrumentation engineering project work complies with statutory and regulatory requirements and company policies and practices;
Providing assistance to ensure all electrical and instrumentation engineering project work is completed on time and within budget;
Assisting with the preparation of preliminary and detailed designs of electrical and instrumentation engineering projects;
Assisting with the preparation of documentation for standards, procedures, safe working practices and monitoring regimes in respect of work on or around electrical and instrumentation engineering projects;
Assisting with the preparation of maintenance planning for all electrical and instrumentation engineering projects; and
Liaison with other department engineers and draftspersons to ensure electrical and instrumentation project goals are met.'''

In [ ]:
data = dmatch(text,data)